In [3]:
import requests

CHUNK_SIZE = 1024
voice_id = "hDDJRL9aTI0hbD6crp4v"
url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
XI_API_KEY = "7b9879f9e2b120bea4cf089b2ae47ef4"

headers = {
  "Accept": "audio/mpeg",
  "Content-Type": "application/json",
  "xi-api-key": XI_API_KEY
}

data = {
  "text": "Hi! My name is Bella, nice to meet you!",
  "model_id": "eleven_monolingual_v1",
  "voice_settings": {
    "stability": 0.5,
    "similarity_boost": 0.5
  }
}

response = requests.post(url, json=data, headers=headers)
with open('output.mp3', 'wb') as f:
    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)



In [5]:
help(response)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

In [ ]:
import requests

# streaming chunk size
CHUNK_SIZE = 1024

XI_API_KEY = "7b9879f9e2b120bea4cf089b2ae47ef4"
VOICE_SAMPLE_PATH1 = "<path-to-file>"
VOICE_SAMPLE_PATH2 = "<path-to-file>"
OUTPUT_PATH = "<path-to-file>"

add_voice_url = "https://api.elevenlabs.io/v1/voices/add"

headers = {
  "Accept": "application/json",
  "xi-api-key": XI_API_KEY
}

data = {
    'name': 'Voice name',
    'labels': '{"accent": "American", "gender": "Female"}',
    'description': 'An old American male voice with a slight hoarseness in his throat. Perfect for news.'
}

files = [
    ('files', ('sample1.mp3', open(VOICE_SAMPLE_PATH1, 'rb'), 'audio/mpeg')),
    ('files', ('sample2.mp3', open(VOICE_SAMPLE_PATH2, 'rb'), 'audio/mpeg'))
]

response = requests.post(add_voice_url, headers=headers, data=data, files=files)
voice_id = response.json()["voice_id"]

# get default voice settings
response = requests.get(
    "https://api.elevenlabs.io/v1/voices/settings/default",
    headers={ "Accept": "application/json" }
).json()
stability, similarity_boost = response["stability"], response["similarity_boost"]

tts_url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}/stream"

headers["Content-Type"] = "application/json"

data = {
  "text": "Some very long text to be read by the voice",
  "model_id": "eleven_monolingual_v1",
  "voice_settings": {
    "stability": stability,
    "similarity_boost": similarity_boost
  }
}

response = requests.post(tts_url, json=data, headers=headers, stream=True)

with open(OUTPUT_PATH, 'wb') as f:
    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

# Retrieve history. It should contain generated sample.
history_url = "https://api.elevenlabs.io/v1/history"

headers = {
  "Accept": "application/json",
  "xi-api-key": XI_API_KEY
}

response = requests.get(history_url, headers=headers)

print(response.text)
